Rand Agent Table with RNN for Time Series: for all agents in a given frame, version 1: doing it for all agents in a scene

#### The way it works: 
* it uses the same multiple parallel series process seen in the rnn notebook for single agents
* but this time, we prepared the dataset table to be done differently
* for ONE GIVEN SCENE, we separate the agent table into their own sub-tables by their track ids, 
* and left outer join them each side by side in a new table called Apis, on the frame-index column
* The rest of Apis are Nan, representing the points in time when a certain track id entity leaves the vision of the ai car.
* We finally apply the same parallel input series process

* The result is bad, we still need to fit random forests some where.

Look at this for more info on multiple parallel series with RNN: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [ ]:
import keras
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
folderpath = "*"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft = pd.read_csv(file)
file = folderpath +  "\\lyftlong\\rand_scenes_table1.csv"
st = pd.read_csv(file)

In [ ]:
at = at.merge(ft[['frame_index', 'scene_index']], on='frame_index')

In [ ]:
atcols = ['centroid_x', 'centroid_y', 'extent_x', 'extent_y', 'extent_z',
          'velocity_x', 'velocity_y', 'yaw', 'track_id','frame_index', 'scene_index']

In [ ]:
atrnn = at[atcols]

In [ ]:
scene_indices = pd.unique(at['scene_index'])

In [ ]:
atrnn1 = atrnn[atrnn.scene_index == scene_indices[0]]

In [ ]:
atrnn1['c'] = np.ones(len(atrnn1))
atrnn1.groupby('frame_index').count()['c'].min()

In [ ]:
agent_parallel_input_series = pd.DataFrame()
apis = agent_parallel_input_series

In [ ]:
# PREPARING THE COLUMNS FOR APIS

In [ ]:
atrnn1s = atrnn1 #.sort_values(['track_id', 'frame_index'])

In [ ]:
# making area:
atrnn1s['area'] = atrnn1s['extent_x'] * atrnn1s['extent_y']

In [ ]:
track_ids = pd.unique(atrnn1s['track_id'])
NUM_ENTITIES = len(track_ids)

In [ ]:
apis_cols = ['centroid_x', 'centroid_y', 'area', 'velocity_x', 'velocity_y', 'frame_index']

In [ ]:
apis_frame_indices = pd.unique(atrnn1['frame_index'])

In [ ]:
apis['frame_index'] = apis_frame_indices

In [ ]:
for i in range(len(track_ids)):
    if i%100 == 0:
        print("{0} of {1}".format(i, len(track_ids)))
    ti = track_ids[i]
    # for each track id, get it's subset in atrnn1s
    atrnn1sti = atrnn1s[atrnn1s.track_id == ti]
    # get these columns only
    atrnn1sti = atrnn1sti[apis_cols]
    # rename the columns by labelling it with it's track id
    atrnn1sti = atrnn1sti.add_prefix('ti{0}_'.format(str(i)))
    # undo apis index's renaming for joining to apis
    atrnn1sti.rename(columns = {"ti{0}_frame_index".format(str(i)): "frame_index"},  
                     inplace = True) 
    # LEFT OUTER JOIN with APIS, atrnn1sti
    apis = pd.merge(apis, atrnn1sti, on='frame_index', how='left')

In [ ]:
apis.shape

In [ ]:
frame_indexer_col_count = 1
assert apis.shape == (248, 5850 + frame_indexer_col_count)

In [ ]:
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [ ]:
apis = apis.drop(['frame_index'], axis = 1)

In [ ]:
apis = apis.fillna(0)

In [ ]:
atrnn1 = apis

In [ ]:
sampcols= ['ti0_centroid_x', 'ti0_centroid_y', 'ti0_area', 'ti0_velocity_x',
       'ti0_velocity_y', 'ti1_centroid_x', 'ti1_centroid_y', 'ti1_area',
       'ti1_velocity_x', 'ti1_velocity_y']

apisrf = apis[sampcols]

In [ ]:
apisrf

In [ ]:
# https://www.pluralsight.com/guides/machine-learning-for-time-series-data-in-python
# Random Forest Regressors

# https://towardsdatascience.com/how-not-to-use-machine-learning-for-time-series-forecasting-avoiding-the-pitfalls-19f9d7adf424
# please look at the cross-correlation to see if data is cross correlated (patternous)

In [ ]:
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
TRAIN_PROPORTION = 0.8
TRAIN_PROPORTION_INDEXER = int(len(apisrf) * 0.8)

In [ ]:
train = apisrf.iloc[:TRAIN_PROPORTION_INDEXER]
test = apisrf.iloc[TRAIN_PROPORTION_INDEXER:]

In [ ]:
target_column_train = ['ti0_velocity_x'] 
predictors_train = list(set(list(train.columns))-set(target_column_train))

In [ ]:
X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

In [ ]:
target_column_test = ['ti0_velocity_x'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

In [ ]:
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [ ]:
model_rf = RandomForestRegressor(n_estimators=5000, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 
pred_train_rf= model_rf.predict(X_train)
print("MSE train", np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print("R2 train", r2_score(y_train, pred_train_rf))

pred_test_rf = model_rf.predict(X_test)
print("MSE test", np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print("R2 test", r2_score(y_test, pred_test_rf))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:

model = GradientBoostingRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
model.fit(X_train, y_train)
# make a single prediction
# row = [0.20543991, -0.97049844, -0.81403429, -0.23842689, -0.60704084, -0.48541492, 0.53113006, 2.01834338, -0.90745243, -1.85859731, -1.02334791, -0.6877744, 0.60984819, -0.70630121, -1.29161497, 1.32385441, 1.42150747, 1.26567231, 2.56569098, -0.11154792]
j = 49

yhat = model.predict([X_test[j]])
# summarize prediction
print('Pred:', yhat[0])
print('Targ:', y_test[j][0])

In [ ]:
# xgboost for regression
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)
# evaluate the model
model = XGBRegressor(objective='reg:squarederror')
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
model = XGBRegressor(objective='reg:squarederror')
model.fit(X, y)
# make a single prediction
row = [2.02220122, 0.31563495, 0.82797464, -0.30620401, 0.16003707, -1.44411381, 0.87616892, -0.50446586, 0.23009474, 0.76201118]
row = asarray(row).reshape((1, len(row)))
yhat = model.predict(row)
print('Prediction: %.3f' % yhat[0])